In [1]:
import pandas as pd
import numpy as np
import os

In [25]:
# ここを適宜変える
csv_filename = "SAMPLE.csv"

df = pd.read_csv(os.path.join("Labels", csv_filename), index_col=0)

In [26]:
df.head()

,patient_id,filename,age,sex,target
0,xxxxxxxxxx,2019_00364.csv,20.0,女性,0
1,xxxxxxxxxx,2019_00217.csv,20.0,女性,0
2,xxxxxxxxxx,2019_00691.csv,20.0,女性,0
3,xxxxxxxxxx,2019_00204.csv,20.0,女性,0
4,yyyyyyyyyy,2019_00384.csv,70.0,女性,1


In [27]:
df_normal = df.query('target == 0')
df_abnormal = df.query('target == 1')

In [28]:
print(df_normal.shape)
print(df_abnormal.shape)

(20, 5)
(16, 5)


In [29]:
def train_valid_test(df):
    df_unique_patient_id = df["patient_id"].value_counts().rename_axis('patient_id').reset_index(name='counts')
    df_rand=pd.concat([df_unique_patient_id, pd.DataFrame( np.random.rand( len( df_unique_patient_id ) ), columns=['rand'] ) ], axis=1 )
    df = pd.merge(df, df_rand, on='patient_id')
    df.loc[df["rand"] > 0.3, 'split'] = 'TRAIN'
    df.loc[(df["rand"] <= 0.3) & (df["rand"] > 0.15), 'split'] = 'VALID'
    df.loc[df["rand"] <= 0.15, 'split'] = 'TEST'
    print(df["split"].value_counts())
    return df

In [30]:
df_normal = train_valid_test(df_normal)
df_normal.head()

TRAIN    11
VALID     6
TEST      3
Name: split, dtype: int64


,patient_id,filename,age,sex,target,counts,rand,split
0,xxxxxxxxxx,2019_00364.csv,20.0,女性,0,4,0.159047,VALID
1,xxxxxxxxxx,2019_00217.csv,20.0,女性,0,4,0.159047,VALID
2,xxxxxxxxxx,2019_00691.csv,20.0,女性,0,4,0.159047,VALID
3,xxxxxxxxxx,2019_00204.csv,20.0,女性,0,4,0.159047,VALID
4,zzzzzzzzzz,2019_00282.csv,30.0,男性,0,3,0.505219,TRAIN


In [31]:
df_abnormal = train_valid_test(df_abnormal)
df_abnormal.head()

VALID    8
TRAIN    6
TEST     2
Name: split, dtype: int64


,patient_id,filename,age,sex,target,counts,rand,split
0,yyyyyyyyyy,2019_00384.csv,70.0,女性,1,3,0.172297,VALID
1,yyyyyyyyyy,2019_00733.csv,70.0,女性,1,3,0.172297,VALID
2,yyyyyyyyyy,2019_00039.csv,70.0,女性,1,3,0.172297,VALID
3,bbbbbbbbbb,2019_00645.csv,80.0,男性,1,2,0.554090,TRAIN
4,bbbbbbbbbb,2019_00041.csv,80.0,男性,1,2,0.554090,TRAIN


In [32]:
df = pd.concat([df_normal, df_abnormal], axis=0 )
df["split"].value_counts()

TRAIN    17
VALID    14
TEST      5
Name: split, dtype: int64

In [33]:
df  = df.drop(columns=["patient_id", "counts", "rand"])
df.head()

,filename,age,sex,target,split
0,2019_00364.csv,20.0,女性,0,VALID
1,2019_00217.csv,20.0,女性,0,VALID
2,2019_00691.csv,20.0,女性,0,VALID
3,2019_00204.csv,20.0,女性,0,VALID
4,2019_00282.csv,30.0,男性,0,TRAIN


In [34]:
df.to_csv(os.path.join('FileLists', 'FileList_{}.csv'.format(csv_filename.replace(".csv", ""))))